In [1]:
import torch

In [2]:
N, W, H = (3, 3, 3)
x = torch.randn([N,W,H])
print(x)
x = x.reshape([N, W*H])
print(x)

tensor([[[-0.0586, -0.0960, -1.1289],
         [ 0.7551,  0.5630,  1.7815],
         [-1.7009, -1.5850, -0.8644]],

        [[ 0.2892,  0.5145,  1.8476],
         [-1.3496, -2.1752,  2.0551],
         [ 0.4155,  1.3952, -0.0387]],

        [[-0.5807, -1.4092,  1.7368],
         [ 1.8485,  1.3379, -0.0839],
         [ 0.1671,  0.7975,  0.9323]]])
tensor([[-0.0586, -0.0960, -1.1289,  0.7551,  0.5630,  1.7815, -1.7009, -1.5850,
         -0.8644],
        [ 0.2892,  0.5145,  1.8476, -1.3496, -2.1752,  2.0551,  0.4155,  1.3952,
         -0.0387],
        [-0.5807, -1.4092,  1.7368,  1.8485,  1.3379, -0.0839,  0.1671,  0.7975,
          0.9323]])


In [3]:
_, indices = torch.sort(x, descending=True, dim=1)
print(indices)

tensor([[5, 3, 4, 0, 1, 8, 2, 7, 6],
        [5, 2, 7, 1, 6, 0, 8, 3, 4],
        [3, 2, 4, 8, 7, 6, 5, 0, 1]])


In [4]:
top_k = 3
top_indices = indices[:, :top_k]
print(top_indices)

tensor([[5, 3, 4],
        [5, 2, 7],
        [3, 2, 4]])


In [5]:
print(x.shape)
print(top_indices.shape)

print(x)
print(top_indices)

torch.Size([3, 9])
torch.Size([3, 3])
tensor([[-0.0586, -0.0960, -1.1289,  0.7551,  0.5630,  1.7815, -1.7009, -1.5850,
         -0.8644],
        [ 0.2892,  0.5145,  1.8476, -1.3496, -2.1752,  2.0551,  0.4155,  1.3952,
         -0.0387],
        [-0.5807, -1.4092,  1.7368,  1.8485,  1.3379, -0.0839,  0.1671,  0.7975,
          0.9323]])
tensor([[5, 3, 4],
        [5, 2, 7],
        [3, 2, 4]])


In [6]:
mask = x.clone()
for i in range(N):
    mask[i, top_indices[i]] = 0

print(x)
print(mask)

tensor([[-0.0586, -0.0960, -1.1289,  0.7551,  0.5630,  1.7815, -1.7009, -1.5850,
         -0.8644],
        [ 0.2892,  0.5145,  1.8476, -1.3496, -2.1752,  2.0551,  0.4155,  1.3952,
         -0.0387],
        [-0.5807, -1.4092,  1.7368,  1.8485,  1.3379, -0.0839,  0.1671,  0.7975,
          0.9323]])
tensor([[-0.0586, -0.0960, -1.1289,  0.0000,  0.0000,  0.0000, -1.7009, -1.5850,
         -0.8644],
        [ 0.2892,  0.5145,  0.0000, -1.3496, -2.1752,  0.0000,  0.4155,  0.0000,
         -0.0387],
        [-0.5807, -1.4092,  0.0000,  0.0000,  0.0000, -0.0839,  0.1671,  0.7975,
          0.9323]])


In [43]:
def generate_attentive_mask(attention_map, top_k):
    """
        Input:
            attention_map   (Tensor) : NxWxH tensor after GAP.
            top_k           (Tensor) : Number of candidates of the most intense points.
        Output:
            mask            (Tensor) : NxWxH tensor for masking attentive regions
            coords          (Tensor) : Normalized coordinates(cx, cy) for masked regions
    """
    N, W, H = attention_map.shape
    x = attention_map.reshape([N, W *H])

    _, indices = torch.sort(x, descending=True, dim=1)

    top_indices = indices[:, :top_k]

    cell_width, cell_height = 1/W, 1/H

    rows, cols = (top_indices//W)/N, (top_indices%W)/N
    cx = cell_width/2 + rows*cell_width
    cy = cell_height/2 + cols*cell_height
    coords = torch.cat((cx, cy), dim=0).T

    print(cx, cy)
    print(coords)

    mask = x.clone()

    for i in range(N):
        mask[i, top_indices[i]] = 0

    mask = mask.reshape([N, W, H])
    # print(mask)
    mask[mask != 0] = 1

    return mask, coords

N, W, H = (1, 2,2)
attention_map = torch.randn([N,W,H])
# print(attention_map)
generate_attentive_mask(attention_map, top_k=3)

tensor([[0.2500, 0.7500, 0.2500]]) tensor([[0.7500, 0.7500, 0.2500]])
tensor([[0.2500, 0.7500],
        [0.7500, 0.7500],
        [0.2500, 0.2500]])


(tensor([[[0., 0.],
          [1., 0.]]]),
 tensor([[0.2500, 0.7500],
         [0.7500, 0.7500],
         [0.2500, 0.2500]]))